# 重新定义大模型和智能体

## 综合例子

In [1]:
# 创建一个工具
from illufly.chat import ChatQwen
from illufly.types import BaseAgent, PromptTemplate, EventBlock

def get_current_weather(location: str):
    """获取城市的天气情况"""
    # return f"{location}今天是晴天。 "
    yield EventBlock("chunk", f"{location}今天是晴天。 ")

## 声明大模型
qwen = ChatQwen(
    tools=[BaseAgent(get_current_weather)],
    knowledge=["这里的兔子是指嫦娥的宠物兔子"],
    memory=[("system", "你是一个作家")]
)

qwen("先查询广州天气，然后结合兔子的主人，帮我写一首关于兔子的四句打油诗", verbose=True)

[INFO] 记住 10 轮对话
[TOOLS_CALL_CHUNK] {"index": 0, "id": "call_96048acc60d043f899fbf3", "type": "function", "function": {"name": "get_current_weather", "arguments": ""}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"arguments": "{\"location\": \""}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"arguments": "广州\"}"}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {}}

[USAGE] {"input_tokens": 184, "output_tokens": 18, "total_tokens": 202}

[FINAL_TOOLS_CALL] [{"index": 0, "id": "call_96048acc60d043f899fbf3", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"广州\"}"}}]
广州今天是晴天。 
[FINAL_TOOL_RESP] 广州今天是晴天。 
好的，广州今天是晴天。接下来我为您创作一首关于兔子的四句打油诗：
小兔蹦跳在绿野，
主人相伴乐无边。
晴空万里风轻扬，
幸福时光共长天。
[USAGE] {"input_tokens": 217, "out

'好的，广州今天是晴天。接下来我为您创作一首关于兔子的四句打油诗：\n小兔蹦跳在绿野，\n主人相伴乐无边。\n晴空万里风轻扬，\n幸福时光共长天。'

In [2]:
qwen.memory

[{'role': 'system', 'content': '你是一个作家'},
 {'role': 'user', 'content': '已知知识：\n这里的兔子是指嫦娥的宠物兔子'},
 {'role': 'assistant', 'content': 'OK'},
 {'role': 'user', 'content': '先查询广州天气，然后结合兔子的主人，帮我写一首关于兔子的四句打油诗'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': 'call_13c5bdfd6bd5410faebd1d',
    'type': 'function',
    'function': {'name': 'get_current_weather',
     'arguments': '{"location": "广州"}'}}]},
 {'tool_call_id': 'call_13c5bdfd6bd5410faebd1d',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '广州今天是晴天。 '},
 {'role': 'assistant',
  'content': '在广州的晴朗一天里，\n小白兔，你欢快地蹦跳，\n嫦娥姐姐在天空微笑着看你，\n阳光下，你的身影如此生动。'}]

## 多轮对话

使用`messages`管理多轮对话的记忆。<br>
你也可以在`chat`函数中通过参数`k`限制希望保留的对话轮次，但这不影响`messages`记录完整的对话过程。

In [1]:
from illufly.chat import FakeLLM, ChatZhipu, ChatQwen
from illufly.io import log, alog

a = ChatQwen()

In [2]:
a.build_chat_memory("hi")

[{'role': 'user', 'content': 'hi'}]

## 回调工具

In [1]:
from illufly.chat import ChatQwen
from illufly.types import BaseAgent

def get_current_weather(location: str):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

qwen = ChatQwen(tools=[BaseAgent(get_current_weather)])
qwen("今天广州天气如何啊")



[TOOLS_CALL_FINAL] [{"index": 0, "id": "call_b1beefa5dde64281b26226", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"广州\"}"}}]
广州今天是晴天。 
广州今天的天气是晴天。


'广州今天的天气是晴天。'

In [8]:
qwen.tools[0].tool_desc

{'type': 'function',
 'function': {'name': 'get_current_weather',
  'description': '获取城市的天气情况',
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string', 'description': ''}},
   'required': ['location']}}}

In [9]:
qwen("这是怎么得出来的？")

我是通过使用天气获取功能查询得到的。我查询了广州今天的天气，反馈的结果是晴天。这就是依据。如果你想查看当时的查询结果，你可以让我再做一次查询。

广州今天是晴天。 
这是通过查询当前天气接口得出的，广州今天的天气是晴天。


'这是通过查询当前天气接口得出的，广州今天的天气是晴天。'

In [10]:
qwen("再看看上海")



上海今天是晴天。 
好的，通过查询当前天气的接口得知，上海今天的天气也是晴天。


'好的，通过查询当前天气的接口得知，上海今天的天气也是晴天。'

## 何时构建新对话

### 指定新对话

### 通过携带 system 角色的消息列表，发起新对话